In [1]:
!pip install --upgrade --quiet langchain langchain-community langchain-groq neo4j langchain_experimental

# Importing libraries

import os
from google.colab import userdata
from langchain_groq import ChatGroq
from langchain_core.documents import Document
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer

# Set up environment variables

NEO4J_URI = userdata.get('NEO4J_URI')
NEO4J_USERNAME = userdata.get('NEO4J_USERNAME')
NEO4J_PASSWORD = userdata.get('NEO4J_PASSWORD')
userdata.get('groq_api_key')

# Initialize Neo4jGraph

graph = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

# Initialize LLM

llm = ChatGroq(groq_api_key=userdata.get('groq_api_key'), model_name='Gemma2-9b-It')

# Define the text and create Document

text = """
Rohit Gurunath Sharma (born 30 April 1987) is an Indian international cricketer who currently plays for and captains the India national cricket team
in Test and One Day International (ODI) matches. Previously, he also captained the team in Twenty20 International (T20I) matches and led India's win
in 2024 ICC Men's T20 World Cup, subsequent to which he retired from T20s in June 2024. The right-handed batsman, who is considered one of the
best batsmen of his generation and one of the greatest opening batters of all time, is known for his timing, elegance, six-hitting abilities and
leadership skills.
Sharma was born on 30 April 1987 into a Telugu-Marathi speaking family in Bansod, Nagpur, Maharashtra, India. His mother, Purnima Sharma,
is from Visakhapatnam, Andhra Pradesh. His father, Gurunath Sharma, worked as a caretaker of a transport firm storehouse. Sharma was raised by
his grandparents and uncles in Borivali because of his father's low income. He would visit his parents, who lived in a single-room house in Dombivli,
only during weekends. He has a younger brother, Vishal Sharma.
Sharma married his longtime girlfriend, Ritika Sajdeh on 13 December 2015 whom he first met in 2008. They have one child, a girl born
on 30 December 2018 named Samaira. Sharma is a practitioner of the meditation technique Sahaj Marg. He practices an eggetarian diet.
Sharma clinched the world record for the highest ever individual score in a one-day international match, with a remarkable 264 against
Sri Lanka at Eden Gardens, Kolkata, on 13 November 2014. He is the only player to have scored three double-centuries in this
form of international cricket. In January 2020, Sharma was named as the ODI Player of the Year by the International Cricket Council
(ICC). During the 2019 World Cup, Sharma became the only batsman to scored five centuries in a single edition of the Cricket World Cup.
"""

document = [Document(page_content=text)]

# Convert Document to Graph

llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(document)

# Print converted graph documents

print("Graph Nodes:", graph_documents[0].nodes)
print("Graph Relationships:", graph_documents[0].relationships)

# Define the Cypher query

cricket_query = """

CREATE (Rohit_Gurunath_Sharma:Person {name: 'Rohit Gurunath Sharma', dob: date('1987-04-30'), batting_style: 'Right-handed', description: 'Indian international cricketer', record: 'Highest ever individual score in a one-day international match with 264 against Sri Lanka on 13 November 2014', accolades: 'ODI Player of the Year 2020 by ICC, Only batsman to score five centuries in a single edition of Cricket World Cup 2019', meditation: 'Practitioner of Sahaj Marg', diet: 'Eggetarian'})
CREATE (India_National_Cricket_Team:Team {name: 'India National Cricket Team'})
CREATE (Visakhapatnam:Place {name: 'Visakhapatnam'})
CREATE (Bansod:Place {name: 'Bansod'})
CREATE (Nagpur:Place {name: 'Nagpur'})
CREATE (Maharashtra:Place {name: 'Maharashtra'})
CREATE (Andhra_Pradesh:Place {name: 'Andhra Pradesh'})
CREATE (Borivali:Place {name: 'Borivali'})
CREATE (Dombivli:Place {name: 'Dombivli'})
CREATE (Kolkata:Place {name: 'Kolkata'})
CREATE (Eden_Gardens:Place {name: 'Eden Gardens'})
CREATE (Sri_Lanka:Team {name: 'Sri Lanka'})
CREATE (Purnima_Sharma:Person {name: 'Purnima Sharma', place: 'Visakhapatnam'})
CREATE (Gurunath_Sharma:Person {name: 'Gurunath Sharma', occupation: 'Caretaker'})
CREATE (Vishal_Sharma:Person {name: 'Vishal Sharma'})
CREATE (Ritika_Sajdeh:Person {name: 'Ritika Sajdeh', relationship: 'Wife', married_on: date('2015-12-13')})
CREATE (Samaira_Sharma:Person {name: 'Samaira Sharma', dob: date('2018-12-30')})

CREATE (Rohit_Gurunath_Sharma)-[:CAPTAIN_OF]->(India_National_Cricket_Team)
CREATE (Rohit_Gurunath_Sharma)-[:BORN_IN]->(Bansod)
CREATE (Rohit_Gurunath_Sharma)-[:RAISED_IN]->(Nagpur)
CREATE (Rohit_Gurunath_Sharma)-[:RAISED_IN]->(Maharashtra)
CREATE (Rohit_Gurunath_Sharma)-[:RAISED_IN]->(Borivali)
CREATE (Rohit_Gurunath_Sharma)-[:VISITS]->(Dombivli)
CREATE (Purnima_Sharma)-[:FROM]->(Visakhapatnam)
CREATE (Rohit_Gurunath_Sharma)-[:MOTHER_OF]->(Purnima_Sharma)
CREATE (Rohit_Gurunath_Sharma)-[:FATHER_OF]->(Gurunath_Sharma)
CREATE (Rohit_Gurunath_Sharma)-[:BROTHER_OF]->(Vishal_Sharma)
CREATE (Rohit_Gurunath_Sharma)-[:MARRIED_TO {since: date('2015-12-13')}]->(Ritika_Sajdeh)
CREATE (Rohit_Gurunath_Sharma)-[:PARENT_OF]->(Samaira_Sharma)
CREATE (Rohit_Gurunath_Sharma)-[:SCORED {score: 264, date: date('2014-11-13')}]->(Eden_Gardens)
CREATE (Rohit_Gurunath_Sharma)-[:AGAINST]->(Sri_Lanka)

"""

# Execute the query and refresh schema

graph.query(cricket_query)
graph.refresh_schema()
print("Updated Graph Schema:", graph.schema)

# Initialize GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(llm, graph=graph, verbose=True)

Graph Nodes: [Node(id='Rohit Gurunath Sharma', type='Person'), Node(id='India National Cricket Team', type='Team'), Node(id='Purnima Sharma', type='Person'), Node(id='Gurunath Sharma', type='Person'), Node(id='Vishal Sharma', type='Person'), Node(id='Ritika Sajdeh', type='Person'), Node(id='Samaira', type='Person'), Node(id='Sri Lanka', type='Team'), Node(id='Eden Gardens', type='Location'), Node(id='Bansod', type='Location'), Node(id='Nagpur', type='Location'), Node(id='Maharashtra', type='Location'), Node(id='India', type='Country'), Node(id='Visakhapatnam', type='Location'), Node(id='Andhra Pradesh', type='Location'), Node(id='Dombivli', type='Location'), Node(id='Borivali', type='Location'), Node(id='International Cricket Council', type='Organization'), Node(id='Kolkata', type='Location')]
Graph Relationships: [Relationship(source=Node(id='Rohit Gurunath Sharma', type='Person'), target=Node(id='India National Cricket Team', type='Team'), type='CAPTAIN'), Relationship(source=Node(id

In [2]:
# Define and execute queries

queries = [
    'Who is Rohit Gurunath Sharma?',
    'Who is Rohit Gurunath Sharma married to?',
    'Who is Rohit Gurunath Sharma child?',
    'Who is Gurunath Sharma?']

for query in queries:
    response = chain.invoke({'query': query})
    print(f"Query: {query}\nResponse: {response}\n")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Rohit Gurunath Sharma"}) RETURN p 

Full Context:
[{'p': {'dob': neo4j.time.Date(1987, 4, 30), 'name': 'Rohit Gurunath Sharma', 'batting_style': 'Right-handed', 'description': 'Indian international cricketer'}}, {'p': {'dob': neo4j.time.Date(1987, 4, 30), 'name': 'Rohit Gurunath Sharma', 'batting_style': 'Right-handed', 'description': 'Indian international cricketer'}}, {'p': {'dob': neo4j.time.Date(1987, 4, 30), 'name': 'Rohit Gurunath Sharma', 'batting_style': 'Right-handed', 'description': 'Indian international cricketer'}}, {'p': {'dob': neo4j.time.Date(1987, 4, 30), 'name': 'Rohit Gurunath Sharma', 'batting_style': 'Right-handed', 'description': 'Indian international cricketer'}}, {'p': {'dob': neo4j.time.Date(1987, 4, 30), 'name': 'Rohit Gurunath Sharma', 'batting_style': 'Right-handed', 'description': 'Indian international cricketer'}}, {'p': {'dob': neo4j.time.Date(1987, 4, 30), 'name': 'Rohi